In [1]:
import numpy as np
import pandas as pd

csv_file_path = '../data/LKA/SriLanka_MinHealth_Dengue incidence_2010_2016.csv'

In [2]:
df = pd.read_csv(csv_file_path, header=None, sep=",")
df.head()

,0,1,2,3,4,5,6,7,8,9,...,75,76,77,78,79,80,81,82,83,84
0,ISO,NAME_0,NaN,RDHS,201001,201002,201003,201004,201005,201006,...,201512,201601,201602,201603,201604,201605,201606,201607,201608,201609
1,LKA,Sri Lanka,LK.CO,Colombo,584,606,294,224,296,700,...,1321,2184,1569,1016,942,693,1459,3163,1478,939
2,LKA,Sri Lanka,LK.GQ,Gampaha,608,623,306,205,295,381,...,496,842,615,260,276,221,555,1625,687,513
3,LKA,Sri Lanka,LK.KT,Kalutara,121,124,119,114,185,294,...,202,321,294,197,312,241,343,683,286,187
4,LKA,Sri Lanka,LK.KY,Kandy,211,174,116,89,114,176,...,170,335,207,93,109,152,504,1215,508,347


In [3]:
def convert_date(raw_date):
    result = '{0}01'.format(raw_date)
    return result

In [4]:
arr = df.as_matrix().T

dates_raw = arr[4:, 0]
dates = [convert_date(x) for x in dates_raw]

places_raw = arr[:4, :]

new_arr = np.array([])

for i in range(1, arr.shape[1]):
    place_data = arr[:4, i]
       
    dengue_data = arr[4:, i]
    for j in range(dengue_data.shape[0]):
        date = dates[j]
        dengue_cases = dengue_data[j]
        if dengue_cases == 'n/a':
            dengue_cases = -1
        
        new_row = np.array(place_data)
        new_row = np.append(new_row, date)
        new_row = np.append(new_row, dengue_cases)
        
        if new_arr.shape[0] == 0:
            new_arr = new_row
        else:
            new_arr = np.vstack((new_arr, new_row))

        
column_names = ['country_iso2_id', 'country_name', 'admin_lvl1_id', 'admin_lvl1_name', 'period_date', 'dengue_total_cases']
new_df = pd.DataFrame(new_arr, columns=column_names)
new_df = new_df.apply(lambda x: pd.to_numeric(x, errors='ignore'))

new_df.head()

,country_iso2_id,country_name,admin_lvl1_id,admin_lvl1_name,period_date,dengue_total_cases
0,LKA,Sri Lanka,LK.CO,Colombo,20100101,584
1,LKA,Sri Lanka,LK.CO,Colombo,20100201,606
2,LKA,Sri Lanka,LK.CO,Colombo,20100301,294
3,LKA,Sri Lanka,LK.CO,Colombo,20100401,224
4,LKA,Sri Lanka,LK.CO,Colombo,20100501,296


In [5]:
new_df.to_csv('lka_minhealth_data_refactored.csv', index=False, header=True)